## Testing event tools

In [1]:
import nltk
import pandas as pd 
import string
import event_tools as events
import ipywidgets as widgets
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from IPython.display import display
from collections import Counter
import plotly.graph_objs as go
from IPython.display import display, Markdown


In [ ]:
# Press button to get top N countries
## Global homepage 
# Map GDELT scores and mention frequencies
# Visualize orgs and individual counts + network* + Phrases 
# Get select news based on phrase and entity counts 
# top_n_countries = events.top_n_bar_chart()

## Country breakdowns 
# Get time series from GDELT + ACLED 
# Visualize orgs and individual counts + network* + Phrases 
# Generate news feed: GDELT + RSS 


t_str1 = widgets.FloatText( description='Strain in %:', value='3' )
b_cs   = widgets.Button( description='Compute Stress' )
t_str2 = widgets.Label(value='Waiting' )

vb=widgets.VBox([t_str1,b_cs,t_str2])

def on_click_compute_stress(b):
    t_str2.value= events.get_content('')

b_cs.on_click(on_click_compute_stress)

display(vb)

In [ ]:
def flat_ents1(ents):
    flat_ents = []
    for ent in ents:
        flat_ents = flat_ents + ent[1]

    return flat_ents


def new_count(field,frame):
    indexed = list(zip(list(range(len(frame[field].tolist()))),frame[field].tolist()))
    all_ = flat_ents1(indexed)
    counted_ = {i:all_.count(i) for i in all_}
    counted = sorted(counted_.items(), key=lambda x:x[1], reverse=True)

    return counted



def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list


def get_summaries(headlines,df):
    summaries = []
    for headline in headlines[0]:
        summaries.append(df.loc[df["Title"] == headline]['Summary'].to_list())

    return flatten_list(summaries)


def top_n_bar_chart(data, n,var_name,vertical):
    if vertical == True:
        y = [k[0] for k in data[1:n]]
        x = [k[1] for k in data[1:n]]
        data = [go.Bar(x=x, y=y,orientation='h')]
        layout = go.Layout(title=f"Top {n} "+var_name,template="plotly_dark")
        fig = go.Figure(data=data, layout=layout)
        fig.show()

    else:
        x = [k[0] for k in data[1:n]]
        y = [k[1] for k in data[1:n]]
        data = [go.Bar(x=x, y=y)]
        layout = go.Layout(title=f"Top {n} "+var_name,template="plotly_dark" )
        fig = go.Figure(data=data, layout=layout)
        fig.show()


def country_headlines(country, df):
    headlines = df.loc[df['country'] == country]['headlines']

    return headlines

def extract_ngrams(article, n):
    # Tokenize the article into words
    tokens = nltk.word_tokenize(article)
    
    # Generate the N-grams
    n_grams = ngrams(tokens, n)
    
    # Count the N-grams
    n_gram_counts = Counter(n_grams)
    
    return n_gram_counts


def clean_stopwords(article):
    # Tokenize the article into words
    tokens = word_tokenize(article)
    
    # Get the English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Filter out the stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Join the filtered tokens back into a string
    cleaned_article = ' '.join(filtered_tokens)
    
    
    return cleaned_article


def aggregate_ngrams(articles, n):

    ngrams = []
    for article in articles:
        a = clean_stopwords(article).translate(str.maketrans('', '', string.punctuation))
        
        for k in extract_ngrams(a,n).keys():
            ngrams.append(k)

    return Counter(ngrams)


def country_ngrams(country,master, n):
    summaries = get_summaries(country,master)
    n_gram_counts = dict(sorted(aggregate_ngrams(summaries,n).items(), key=lambda item: item[1],reverse=True))

    return n_gram_counts

def get_entities(headlines, df, type_):
    ents_nested = []
    ents = []
    
    for headline in headlines[0]:
        ents_nested.append(df.loc[df["Title"] == headline][type_].to_list())
    
    for li in ents_nested: 
        for li2 in li:
            for e in li2: 
                ents.append(e)

    return (sorted(Counter(ents).items(),key=lambda item: item[1],reverse=True),ents_nested)



In [ ]:

test_set = pd.read_json('aggregated_dataNewTest2.json')
test_master = pd.read_json('output.json',encoding='utf-8')

all_countries = list(zip(test_set['country'].to_list(),test_set['mentions'].to_list()))
counted_countries = sorted(all_countries, key=lambda x:x[1], reverse=True)

top_n_bar_chart(counted_countries,25,"Associated countries",vertical=True)


In [ ]:
country_ = "Afghanistan"
k = 25
n = 3

country = country_headlines(country_ ,test_set).to_list()
summaries = get_summaries(country,test_master)


n_gram_counts = dict(sorted(aggregate_ngrams(summaries,n).items(), key=lambda item: item[1],reverse=True))

individuals = get_entities(country,test_master,'Associated Individuals')
orgs = get_entities(country,test_master,'Associated Organizations')

top_n_bar_chart(individuals[0],k,"Associated individuals",vertical=True)
top_n_bar_chart(orgs[0],k,"Associated Organizations",vertical=True)

for a in (n_gram_counts.items()):
    if a[1] > 1:
        print(" ".join(a[0])+ ": "+str(a[1]))

for a in country[0]:
    print(a)
